<h4>Solution-1: Reading Employee data in employeeDF and sampleDF dataframe from here: https://www.briandunning.com/sample-data/us-500.zip - Done</h4>

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window

# Laoding sample data
empDF = spark.read.csv("dbfs:/FileStore/tables/maersk/data/us_500.csv", header=True, inferSchema=True)
sampleDF = spark.read.csv("dbfs:/FileStore/tables/maersk/data/us_500.csv", header=True, inferSchema=True)

empDF.show()

empDF.printSchema()

+----------+---------+--------------------+--------------------+-------------+--------------+-----+-----+------------+------------+--------------------+--------------------+
first_name|last_name| company_name| address| city| county|state| zip| phone1| phone2| email| web|
+----------+---------+--------------------+--------------------+-------------+--------------+-----+-----+------------+------------+--------------------+--------------------+
 James| Butt| Benton, John B Jr| 6649 N Blue Gum St| New Orleans| Orleans| LA|70116|504-621-8927|504-845-1427| jbutt@gmail.com|http://www.benton...|
 Josephine| Darakjy|Chanay, Jeffrey A...| 4 B Blue Ridge Blvd| Brighton| Livingston| MI|48116|810-292-9388|810-374-9840|josephine_darakjy...|http://www.chanay...|
 Art| Venere| Chemel, James L Cpa|8 W Cerritos Ave #54| Bridgeport| Gloucester| NJ| 8014|856-636-8749|856-264-4130| art@venere.org|http://www.chemel...|
 Lenna| Paprocki|Feltz Printing Se...| 639 Main St| Anchorage| Anchorage| AK|99501|907-385-4412|907-921-2010|lpaprocki@hotmail...|http://www.feltzp...|
 Donette| Foller| Printing Dimensions| 34 Center St| Hamilton| Butler| OH|45011|513-570-1893|513-549-4561|donette.foller@co...|http://www.printi...|
 Simona| Morasca| Chapman, Ross E Esq| 3 Mcauley Dr| Ashland| Ashland| OH|44805|419-503-2484|419-800-6759| simona@morasca.com|http://www.chapma...|
 Mitsue| Tollner| Morlong Associates| 7 Eads St| Chicago| Cook| IL|60632|773-573-6914|773-924-8565|mitsue_tollner@ya...|http://www.morlon...|
 Leota| Dilliard| Commercial Press| 7 W Jackson Blvd| San Jose| Santa Clara| CA|95111|408-752-3500|408-813-1105| leota@hotmail.com|http://www.commer...|
 Sage| Wieser|Truhlar And Truhl...| 5 Boston Ave #88| Sioux Falls| Minnehaha| SD|57105|605-414-2147|605-794-4895| sage_wieser@cox.net|http://www.truhla...|
 Kris| Marrier|King, Christopher...|228 Runamuck Pl #...| Baltimore|Baltimore City| MD|21224|410-655-8723|410-804-4694| kris@gmail.com|http://www.kingch...|
 Minna| Amigon| Dorl, James J Esq| 2371 Jerrold Ave| Kulpsville| Montgomery| PA|19443|215-874-1229|215-422-8694|minna_amigon@yaho...|http://www.dorlja...|
 Abel| Maclead| Rangoni Of Florence| 37275 St Rt 17m M|Middle Island| Suffolk| NY|11953|631-335-3414|631-677-3675| amaclead@gmail.com|http://www.rangon...|
 Kiley|Caldarera| Feiner Bros| 25 E 75th St #69| Los Angeles| Los Angeles| CA|90034|310-498-5651|310-254-3084|kiley.caldarera@a...|http://www.feiner...|
 Graciela| Ruta|Buckley Miller & ...|98 Connecticut Av...|Chagrin Falls| Geauga| OH|44023|440-780-8425|440-579-7763| gruta@cox.net|http://www.buckle...|
 Cammy| Albares|Rousseaux, Michae...| 56 E Morehead St| Laredo| Webb| TX|78045|956-537-6195|956-841-7216| calbares@gmail.com|http://www.rousse...|
 Mattie| Poquette|Century Communica...| 73 State Road 434 E| Phoenix| Maricopa| AZ|85013|602-277-4385|602-953-6360| mattie@aol.com|http://www.centur...|
 Meaghan| Garufi| Bolton, Wilbur Esq| 69734 E Carrillo St| Mc Minnville| Warren| TN|37110|931-313-9635|931-235-7959| meaghan@hotmail.com|http://www.bolton...|
 Gladys| Rim|T M Byxbee Compan...|322 New Horizon Blvd| Milwaukee| Milwaukee| WI|53207|414-661-9598|414-377-2880| gladys.rim@rim.org|http://www.tmbyxb...|
 Yuki| Whobrey|Farmers Insurance...| 1 State Route 27| Taylor| Wayne| MI|48180|313-288-7937|313-341-4470|yuki_whobrey@aol.com|http://www.farmer...|
 Fletcher| Flosi|Post Box Services...| 394 Manchester Blvd| Rockford| Winnebago| IL|61109|815-828-2147|815-426-5657|fletcher.flosi@ya...|http://www.postbo...|
+----------+---------+--------------------+--------------------+-------------+--------------+-----+-----+------------+------------+--------------------+--------------------+
only showing top 20 rows

root
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- company_name: string (nullable = true)
-- address: string (nullable = true)
-- city: string (nullable = true)
-- county: string (nullable = true)
-- state: string (nullable = true)
-- zip: integer (nullable 

<h4>Solution-2: Using this sampleDF dataframe create 100X more data points and store it into employeeDF ( this will have 50,000 rows ) - Done</h4>

In [0]:
# Re-sampling the data
for i in range(99):
  empDF = empDF.union(sampleDF)

print(empDF.count())


50000

<h4>Solution-3: The company has to plan a vaccination drive and the priority will be based on employee population in a particular city - Done <br/> Solution-4: Create a dataframe of CityEmployeeDensity, the 1st city will be the one with maximum number of employees - Done</h4>

In [0]:
from pyspark.sql.types import IntegerType, DateType

cityEmployeesDF = empDF.select('city').groupBy('city').agg(f.count('city').alias('total_employees')).orderBy(f.desc('total_employees'))

cityEmployeesSpec = Window.orderBy(f.desc('total_employees'))

cityEmployeeDensityDF = cityEmployeesDF.withColumn('vaccine_city_order', f.row_number().over(cityEmployeesSpec)).withColumnRenamed('city', 'employee_city')

cityEmployeeDensityDF.show()

+-------------+---------------+------------------+
employee_city|total_employees|vaccine_city_order|
+-------------+---------------+------------------+
 New York| 1400| 1|
 Philadelphia| 800| 2|
 Chicago| 700| 3|
 Miami| 600| 4|
 Phoenix| 500| 5|
San Francisco| 500| 6|
 Milwaukee| 500| 7|
 Gardena| 500| 8|
 Baltimore| 500| 9|
 Orlando| 500| 10|
 Anchorage| 400| 11|
 Los Angeles| 400| 12|
 Indianapolis| 400| 13|
 Atlanta| 400| 14|
 Brooklyn| 400| 15|
 San Jose| 400| 16|
 Providence| 400| 17|
 Abilene| 400| 18|
 Denver| 400| 19|
 Worcester| 300| 20|
+-------------+---------------+------------------+
only showing top 20 rows

<h4>Solution-5: Create new dataframe by name "VaccinationDrivePlan" with all columns from employeeDF and additional column "Sequence" - Done <br/> Solution-6: In Sequence Column populate the value from the CityEmployeeDensity Dataframe. Print this dataframe - Done</h4>

In [0]:
vaccinationDrivePlanDF = empDF.join(f.broadcast(cityEmployeeDensityDF), empDF.city == cityEmployeeDensityDF.employee_city, "inner").drop('total_employees').withColumnRenamed('vaccine_city_order', 'sequence').drop('employee_city').orderBy('sequence')

# print(vaccinationDrivePlanDF.count())
vaccinationDrivePlanDF.show()

+----------+---------+--------------------+--------------------+--------+--------+-----+-----+------------+------------+--------------------+--------------------+--------+
first_name|last_name| company_name| address| city| county|state| zip| phone1| phone2| email| web|sequence|
+----------+---------+--------------------+--------------------+--------+--------+-----+-----+------------+------------+--------------------+--------------------+--------+
 Derick| Dhamer|Studer, Eugene A Esq| 87163 N Main Ave|New York|New York| NY|10013|212-304-4515|212-225-9676| ddhamer@cox.net|http://www.studer...| 1|
 Mirta| Mallett|Stephen Kennerly ...| 7 S San Marcos Rd|New York|New York| NY|10004|212-870-1286|212-745-6948|mirta_mallett@gma...|http://www.stephe...| 1|
 Willow| Kusko| U Pull It| 90991 Thorburn Ave|New York|New York| NY|10011|212-582-4976|212-934-5167| wkusko@yahoo.com|http://www.upulli...| 1|
 Fausto|Agramonte|Marriott Hotels R...| 5 Harrison Rd|New York|New York| NY|10038|212-313-1783|212-778-3063|fausto_agramonte@...|http://www.marrio...| 1|
 Alishia| Sergi|Milford Enterpris...|2742 Distribution...|New York|New York| NY|10025|212-860-1579|212-753-2740| asergi@gmail.com|http://www.milfor...| 1|
 Fausto|Agramonte|Marriott Hotels R...| 5 Harrison Rd|New York|New York| NY|10038|212-313-1783|212-778-3063|fausto_agramonte@...|http://www.marrio...| 1|
 Jose| Stockham|Tri State Refuele...| 128 Bransten Rd|New York|New York| NY|10011|212-675-8570|212-569-4233| jose@yahoo.com|http://www.trista...| 1|
 Alishia| Sergi|Milford Enterpris...|2742 Distribution...|New York|New York| NY|10025|212-860-1579|212-753-2740| asergi@gmail.com|http://www.milfor...| 1|
 Willow| Kusko| U Pull It| 90991 Thorburn Ave|New York|New York| NY|10011|212-582-4976|212-934-5167| wkusko@yahoo.com|http://www.upulli...| 1|
 Jose| Stockham|Tri State Refuele...| 128 Bransten Rd|New York|New York| NY|10011|212-675-8570|212-569-4233| jose@yahoo.com|http://www.trista...| 1|
 Mirta| Mallett|Stephen Kennerly ...| 7 S San Marcos Rd|New York|New York| NY|10004|212-870-1286|212-745-6948|mirta_mallett@gma...|http://www.stephe...| 1|
 Derick| Dhamer|Studer, Eugene A Esq| 87163 N Main Ave|New York|New York| NY|10013|212-304-4515|212-225-9676| ddhamer@cox.net|http://www.studer...| 1|
 Cyril| Daufeldt|Galaxy Internatio...| 3 Lawton St|New York|New York| NY|10013|212-745-8484|212-422-5427|cyril_daufeldt@da...|http://www.galaxy...| 1|
 Jess| Chaffins|New York Public L...| 18 3rd Ave|New York|New York| NY|10016|212-510-4633|212-428-9538|jess.chaffins@cha...|http://www.newyor...| 1|
 Ozell| Shealy| Silver Bros Inc| 8 Industry Ln|New York|New York| NY|10002|212-332-8435|212-880-8865| oshealy@hotmail.com|http://www.silver...| 1|
 Layla| Springe| Chadds Ford Winery| 229 N Forty Driv|New York|New York| NY|10011|212-260-3151|212-253-7448|layla.springe@cox...|http://www.chadds...| 1|
 Brock| Bolognia| Orinda News| 4486 W O St #1|New York|New York| NY|10003|212-402-9216|212-617-5063| bbolognia@yahoo.com|http://www.orinda...| 1|
 Justine| Mugnolo| Evans Rule Company| 38062 E Main St|New York|New York| NY|10048|212-304-9225|212-311-6377| jmugnolo@yahoo.com|http://www.evansr...| 1|
 Tawna| Buvens|H H H Enterprises...|3305 Nabell Ave #679|New York|New York| NY|10009|212-674-9610|212-462-9157| tawna@gmail.com|http://www.hhhent...| 1|
 Haydee| Denooyer|Cleaning Station Inc| 25346 New Rd|New York|New York| NY|10016|212-792-8658|212-782-3493|hdenooyer@denooye...|http://www.cleani...| 1|
+----------+---------+--------------------+--------------------+--------+--------+-----+-----+------------+------------+--------------------+--------------------+--------+
only showing top 20 rows

<h4>Solution-7: Add date column, only 100 vaccinations are happening a day - Done <br/> Solution-8: Create a Final Vaccination Schedule with a plan to vaccinate 100 employees in a day - Done </h4>

In [0]:
import sys

city_spec = Window.orderBy(f.desc('days_taken'))

vaccinationScheduledDF = cityEmployeeDensityDF.withColumn('start_date', f.current_date()).withColumn('days_taken', (f.floor(f.col('total_employees') / 100))).withColumn('cumsum', f.sum(f.col('days_taken')).over(Window.partitionBy().orderBy().rowsBetween(-sys.maxsize, 0))).withColumn('cum_date', f.expr("date_add(start_date, cumsum)")).withColumn('start_date2', f.lag('cum_date', 1).over(city_spec)).withColumn('schedule_date', f.when(f.col('start_date2').isNull(), f.current_date()).otherwise(f.col('start_date2'))).drop('start_date', 'start_date2').select('employee_city', 'total_employees', 'vaccine_city_order', 'schedule_date', 'days_taken').withColumnRenamed('vaccine_city_order', 'sequence')

vaccinationScheduledDF.show()
# vaccinationScheduleDF.count()


+-------------+---------------+--------+-------------+----------+
employee_city|total_employees|sequence|schedule_date|days_taken|
+-------------+---------------+--------+-------------+----------+
 New York| 1400| 1| 2021-06-21| 14|
 Philadelphia| 800| 2| 2021-07-05| 8|
 Chicago| 700| 3| 2021-07-13| 7|
 Miami| 600| 4| 2021-07-20| 6|
 Phoenix| 500| 5| 2021-07-26| 5|
San Francisco| 500| 6| 2021-07-31| 5|
 Milwaukee| 500| 7| 2021-08-05| 5|
 Gardena| 500| 8| 2021-08-10| 5|
 Baltimore| 500| 9| 2021-08-15| 5|
 Orlando| 500| 10| 2021-08-20| 5|
 Anchorage| 400| 11| 2021-08-25| 4|
 Los Angeles| 400| 12| 2021-08-29| 4|
 Indianapolis| 400| 13| 2021-09-02| 4|
 Atlanta| 400| 14| 2021-09-06| 4|
 Brooklyn| 400| 15| 2021-09-10| 4|
 San Jose| 400| 16| 2021-09-14| 4|
 Providence| 400| 17| 2021-09-18| 4|
 Abilene| 400| 18| 2021-09-22| 4|
 Denver| 400| 19| 2021-09-26| 4|
 Worcester| 300| 20| 2021-09-30| 3|
+-------------+---------------+--------+-------------+----------+
only showing top 20 rows

<h4>Solution-9: Print the final report showing in how many day the vaccination drive is completed per city - Done <br/></h4>
<h4> Here I have appended all 3 columns </h4>

In [0]:
# Final Vaccination Plan
scheduleDF = vaccinationScheduleDF.select('employee_city', 'schedule_date')

finalVaccinationScheduledDF = vaccinationDrivePlanDF.join(f.broadcast(scheduleDF), vaccinationDrivePlanDF.city == scheduleDF.employee_city, "inner").drop('employee_city')

finalVaccinationScheduledDF.show()
print(finalVaccinationScheduledDF.count())

+----------+---------+--------------------+--------------------+--------+--------+-----+-----+------------+------------+--------------------+--------------------+--------+-------------+
first_name|last_name| company_name| address| city| county|state| zip| phone1| phone2| email| web|sequence|schedule_date|
+----------+---------+--------------------+--------------------+--------+--------+-----+-----+------------+------------+--------------------+--------------------+--------+-------------+
 Willow| Kusko| U Pull It| 90991 Thorburn Ave|New York|New York| NY|10011|212-582-4976|212-934-5167| wkusko@yahoo.com|http://www.upulli...| 1| 2021-06-21|
 Alishia| Sergi|Milford Enterpris...|2742 Distribution...|New York|New York| NY|10025|212-860-1579|212-753-2740| asergi@gmail.com|http://www.milfor...| 1| 2021-06-21|
 Jose| Stockham|Tri State Refuele...| 128 Bransten Rd|New York|New York| NY|10011|212-675-8570|212-569-4233| jose@yahoo.com|http://www.trista...| 1| 2021-06-21|
 Brock| Bolognia| Orinda News| 4486 W O St #1|New York|New York| NY|10003|212-402-9216|212-617-5063| bbolognia@yahoo.com|http://www.orinda...| 1| 2021-06-21|
 Tawna| Buvens|H H H Enterprises...|3305 Nabell Ave #679|New York|New York| NY|10009|212-674-9610|212-462-9157| tawna@gmail.com|http://www.hhhent...| 1| 2021-06-21|
 Ozell| Shealy| Silver Bros Inc| 8 Industry Ln|New York|New York| NY|10002|212-332-8435|212-880-8865| oshealy@hotmail.com|http://www.silver...| 1| 2021-06-21|
 Layla| Springe| Chadds Ford Winery| 229 N Forty Driv|New York|New York| NY|10011|212-260-3151|212-253-7448|layla.springe@cox...|http://www.chadds...| 1| 2021-06-21|
 Fausto|Agramonte|Marriott Hotels R...| 5 Harrison Rd|New York|New York| NY|10038|212-313-1783|212-778-3063|fausto_agramonte@...|http://www.marrio...| 1| 2021-06-21|
 Justine| Mugnolo| Evans Rule Company| 38062 E Main St|New York|New York| NY|10048|212-304-9225|212-311-6377| jmugnolo@yahoo.com|http://www.evansr...| 1| 2021-06-21|
 Haydee| Denooyer|Cleaning Station Inc| 25346 New Rd|New York|New York| NY|10016|212-792-8658|212-782-3493|hdenooyer@denooye...|http://www.cleani...| 1| 2021-06-21|
 Mirta| Mallett|Stephen Kennerly ...| 7 S San Marcos Rd|New York|New York| NY|10004|212-870-1286|212-745-6948|mirta_mallett@gma...|http://www.stephe...| 1| 2021-06-21|
 Derick| Dhamer|Studer, Eugene A Esq| 87163 N Main Ave|New York|New York| NY|10013|212-304-4515|212-225-9676| ddhamer@cox.net|http://www.studer...| 1| 2021-06-21|
 Cyril| Daufeldt|Galaxy Internatio...| 3 Lawton St|New York|New York| NY|10013|212-745-8484|212-422-5427|cyril_daufeldt@da...|http://www.galaxy...| 1| 2021-06-21|
 Jess| Chaffins|New York Public L...| 18 3rd Ave|New York|New York| NY|10016|212-510-4633|212-428-9538|jess.chaffins@cha...|http://www.newyor...| 1| 2021-06-21|
 Willow| Kusko| U Pull It| 90991 Thorburn Ave|New York|New York| NY|10011|212-582-4976|212-934-5167| wkusko@yahoo.com|http://www.upulli...| 1| 2021-06-21|
 Alishia| Sergi|Milford Enterpris...|2742 Distribution...|New York|New York| NY|10025|212-860-1579|212-753-2740| asergi@gmail.com|http://www.milfor...| 1| 2021-06-21|
 Jose| Stockham|Tri State Refuele...| 128 Bransten Rd|New York|New York| NY|10011|212-675-8570|212-569-4233| jose@yahoo.com|http://www.trista...| 1| 2021-06-21|
 Brock| Bolognia| Orinda News| 4486 W O St #1|New York|New York| NY|10003|212-402-9216|212-617-5063| bbolognia@yahoo.com|http://www.orinda...| 1| 2021-06-21|
 Tawna| Buvens|H H H Enterprises...|3305 Nabell Ave #679|New York|New York| NY|10009|212-674-9610|212-462-9157| tawna@gmail.com|http://www.hhhent...| 1| 2021-06-21|
 Ozell| Shealy| Silver Bros Inc| 8 Industry Ln|New York|New York| NY|10002|212-332-8435|212-880-8865| oshealy@hotmail.com|http://www.silver...| 1| 2021-06-21|
+----------+---------+--------------------+--------------------+--------+--------+-----+-----+------------+------------+--------------------+--------------------+--------+-------------+
only showing top 20 rows

50000